In [1]:
# Merging sales data with Census income

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline

In [23]:
income_df = pd.read_csv('x19_income.csv', 
                        usecols=['GEOID','B19001e1','B19001e2','B19001e3','B19001e4','B19001e5','B19001e6','B19001e7','B19001e8','B19001e9','B19001e10','B19001e11','B19001e12','B19001e13','B19001e14','B19001e15','B19001e16','B19001e17',
                                 'B19019e1','B19019e2','B19019e3','B19019e4','B19019e5','B19019e6','B19019e7','B19019e8',
                                 'B19049e1','B19049e2','B19049e3','B19049e4','B19049e5'])
income_df.columns=['GEOID','hhinctotalhh','hhinclt10k','hhinc10-15k','hhinc15-20k','hhinc20-25k','hhinc25-30k','hhinc30-35k','hhinc35-40k','hhinc40-45k','hhinc45-50k','hhinc50-60k','hhinc60-75k','hhinc75-100k','hhinc100-125k','hhinc125-150k','hhinc150-200k','hhincgt200k','medhhincbysizetot','medhhincbysize1p','medhhincbysize2p','medhhincbysize3p','medhhincbysize4p','medhhincbysize5p','medhhincbysize6p','medhhincbysize7p','medhhincbyagetotal','medhhincbyagelt25','medhhincbyage25-44','medhhincbyage45-64','medhhincbyagegt65']
## NEED TO DEAL WITH ISSUE WHERE DATA IS SUPPRESSED BY REMOVING ROWS WITH ZEROS
### SUCH AS GEOID 86000US01003 (Amherst, MA)
income_df = income_df.set_index('GEOID')
income_df.head()

,hhinctotalhh,hhinclt10k,hhinc10-15k,hhinc15-20k,hhinc20-25k,hhinc25-30k,hhinc30-35k,hhinc35-40k,hhinc40-45k,hhinc45-50k,...,medhhincbysize3p,medhhincbysize4p,medhhincbysize5p,medhhincbysize6p,medhhincbysize7p,medhhincbyagetotal,medhhincbyagelt25,medhhincbyage25-44,medhhincbyage45-64,medhhincbyagegt65
GEOID,,,,,,,,,,,,,,,,,,,,,
86000US01001,7056,494,259,355,341,298,325,335,210,381,...,75128,83480,99549,128295,96413,58733,37533,68766,79127,29383
86000US01002,9248,912,707,570,352,543,377,335,234,270,...,59777,53688,92813,34306,197143,54422,21094,48378,102500,69452
86000US01003,16,0,0,0,0,0,0,0,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86000US01005,1937,61,42,88,113,99,88,40,19,24,...,83274,93500,83750,NaN,193750,68644,NaN,98000,76667,30265
86000US01007,5798,251,190,371,213,399,183,180,154,257,...,102337,105000,74788,98657,NaN,71875,21016,71976,89867,36173


In [30]:
# GET SALES DATA
sales_csv = "/Users/rajrsingh/Box Sync/CDS/Developer Advocacy/Open Data/sales2009/SalesJan2009.csv"
sales_df = pd.read_csv(sales_csv, dtype={"US Zip": np.str}, usecols=['Product','Price','US Zip','Country'])
# Just get US sales for a single product and get rid of the bad data where price is '13,000'
sales_df = sales_df[ (sales_df.Product == 'Product1') 
                      & (sales_df.Country == 'United States')
                      & (sales_df.Price != '13,000')]
sales_df.drop(['Product','Country'],inplace=True,axis=1)
sales_df['Price'] = pd.to_numeric(sales_df['Price'])
#sales_df['SALE'] = 1
sales_df['GEOID'] = "86000US" + sales_df['US Zip']
#sales_df.sort_values('Price',ascending=False)

# Simplify the data down to number of sales per zip
sales_df.drop(['Price','US Zip'],inplace=True,axis=1)
#sales_census = sales_df['GEOID'].value_counts().to_frame('Sales')
#sales_census['GEOID'] = sales_census.index
sales_counts = sales_df['GEOID'].value_counts()
sales_counts.name = 'Sales'
sales_counts.head(4)

86000US10007    9
86000US77002    7
86000US98104    5
86000US60608    4
Name: Sales, dtype: int64

In [31]:
# Merge sales to census

cs = income_df.join(sales_counts, how='right')
cs.describe()

,hhinctotalhh,hhinclt10k,hhinc10-15k,hhinc15-20k,hhinc20-25k,hhinc25-30k,hhinc30-35k,hhinc35-40k,hhinc40-45k,hhinc45-50k,...,medhhincbysize4p,medhhincbysize5p,medhhincbysize6p,medhhincbysize7p,medhhincbyagetotal,medhhincbyagelt25,medhhincbyage25-44,medhhincbyage45-64,medhhincbyagegt65,Sales
count,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,305.000000,...,296.000000,286.000000,261.000000,203.000000,304.000000,252.000000,304.000000,304.000000,303.000000,306.000000
mean,9689.075410,586.914754,418.908197,396.127869,398.442623,379.354098,396.888525,354.878689,376.780328,332.967213,...,108438.131757,106594.297203,105497.291188,108668.763547,76178.167763,35239.150794,82372.003289,89658.562500,50312.650165,1.303922
std,5966.800159,652.824314,449.423818,370.139270,351.984324,329.388967,353.064200,286.348245,288.884915,250.158066,...,50532.725214,57062.155825,59154.165385,58559.593898,36094.736528,20312.395534,39541.968909,43695.095669,24343.014565,0.862203
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,14688.000000,7457.000000,2499.000000,9919.000000,16069.000000,2499.000000,23387.000000,12819.000000,9739.000000,1.000000
25%,5193.000000,187.000000,127.000000,136.000000,156.000000,131.000000,145.000000,143.000000,141.000000,146.000000,...,72669.000000,62892.000000,64320.000000,66068.500000,50307.000000,21714.000000,53302.500000,59379.250000,34128.000000,1.000000
50%,9239.000000,381.000000,278.000000,301.000000,294.000000,293.000000,307.000000,304.000000,318.000000,277.000000,...,102181.000000,97962.000000,90758.000000,100046.000000,69855.500000,31553.500000,75870.000000,82258.000000,46140.000000,1.000000
75%,13468.000000,761.000000,564.000000,538.000000,559.000000,525.000000,550.000000,486.000000,520.000000,473.000000,...,136833.500000,137992.500000,138750.000000,137902.000000,95624.500000,45303.750000,101802.250000,114105.000000,61451.000000,1.000000
max,35551.000000,5371.000000,3381.000000,2671.000000,2183.000000,1913.000000,2706.000000,1881.000000,1950.000000,1337.000000,...,250001.000000,250001.000000,250001.000000,250001.000000,216037.000000,145556.000000,250001.000000,250001.000000,172143.000000,9.000000
